In [30]:
print("Test compiler/kernel")

Test compiler/kernel


In [31]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
# from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
import os
import time


## Extracted data

In [32]:
# Extract the data from PDF
def load_pdf(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )
    documents = loader.load()

    return documents

In [33]:
extracted_data = load_pdf("data/")

In [34]:
# extracted_data

## Create Text Chunks

In [35]:
# Create text chunks
def text_split(extracted_data): 
    # Initialize a RecursiveCharacterTextSplitter with specified chunk size and overlap
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)

    # Split the documents in 'extracted_data' into text chunks
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [36]:
text_chunks = text_split(extracted_data)
print("Length of my chunks: ", len(text_chunks))

Length of my chunks:  6970


## Download Embedding model

In [37]:
# Download emdeddig model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    
    return embeddings

In [38]:
embeddings = download_hugging_face_embeddings()

In [39]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [40]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [41]:
# query_result

In [59]:
# Initialise the pinecone
pinecone.Pinecone(
   api_key=os.getenv("PINECONE_API_KEY"),
   environment=os.getenv("PINECONE_ENV"),  
)

index_name="medical-chat-bot"


In [61]:
# Creating Embedding for each of the Text Chunks & storing
docsearch = PineconeVectorStore(
    [t.page_content for t in text_chunks], 
    embeddings, 
    index_name=index_name
) 

In [50]:
docsearch
# vectorstore

In [62]:
# If we already have an index we can load it like this
docsearch=Pinecone.from_existing_index(index_name, embeddings)

query = "Where are Allergies"

docs = docsearch.similarity_search(query, k=3)
                                   
print(docs)

TypeError: __init__() missing 1 required positional argument: 'host'

In [63]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [64]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [117]:
llm=CTransformers(
    model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
    model_type="llama",
    config={'max_new_tokens':512,
            'temperature':0.8}
)

In [ ]:
qa=RetrievalQA.from_chain_type(
    llm=llm,
    retriever=docsearch.as_retriever(search_kwargs={'k':2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [ ]:
while True:
    user_input=input(f"Input Prompt: ")
    result=qa({"query": user_input})
    print("Response: ", result["result"])